In [1]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=3
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p]
f=sum(vecf)

g=[1.0-sum(x[I[j]].^2) for j in 1:p]
J=[j:j for j in 1:p]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=0#ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

#=randx=[2*rand(length(I[j])).-1 for j in 1:p]# create a feasible solution
randx=[randx[j]/norm(randx[j]) for j in 1:p]=#

randx=2*rand(Float64,n).-1

for j in 1:p
    randx[I[j]]=randx[I[j]]./norm(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=2
Number of equality constraints: l=0
Relaxed order: k=2


In [2]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

  likely near /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/solvers/ProximalBundleMethod/ProximalMethod.jl:391


In [3]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k;EigAlg="Arpack",maxit=1e10,tol=1e-3,UseEq=false)

------------------------------------------------------


  likely near /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/solvers/ProximalBundleMethod/ProximalMethod.jl:391


The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------


MethodError: MethodError: no method matching get_indcons(::Int64, ::Array{SparseMatrixCSC{UInt64,Ti} where Ti<:Integer,1}, ::Array{Array{UInt16,1},1}, ::Int64, ::Array{Int64,1}; assign="all")
Closest candidates are:
  get_indcons(::Int64, !Matched::SparseMatrixCSC{UInt64,Ti} where Ti<:Integer, ::Array{Array{UInt16,1},1}, ::Int64, !Matched::Array{Array{Int64,1},1}; assign) at /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/src/pop/sparse_pop/basic_func_sparse.jl:3

In [4]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k;EigAlg="Arpack",tol=1e-3,UseEq=false)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------


  likely near /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/solvers/ProximalBundleMethod/ProximalMethod.jl:391


MethodError: MethodError: no method matching get_indcons(::Int64, ::Array{SparseMatrixCSC{UInt64,Ti} where Ti<:Integer,1}, ::Array{Array{UInt16,1},1}, ::Int64, ::Array{Int64,1}; assign="all")
Closest candidates are:
  get_indcons(::Int64, !Matched::SparseMatrixCSC{UInt64,Ti} where Ti<:Integer, ::Array{Array{UInt16,1},1}, ::Int64, !Matched::Array{Array{Int64,1},1}; assign) at /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/src/pop/sparse_pop/basic_func_sparse.jl:3

In [5]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 100             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 6               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 